In [12]:
#pre processing the data
from sklearn.model_selection import train_test_split #used to split the data into training and evaluation
from tensorflow.keras.utils import to_categorical #used to converts data to one hat encoding
import numpy as np
import os

actions = np.array(['hello','how_are_you','good','Bad','Thank_you'])
data_path = os.path.join('mpdata') #path for exported data
label_map = {label:num for num,label in enumerate(actions)} #map the actions to enumeration

In [13]:
label_map

{'hello': 0, 'how_are_you': 1, 'good': 2, 'Bad': 3, 'Thank_you': 4}

In [14]:
#getting back the data
no_sequences = 30
len_sequence = 30

sequences,labels = [],[]
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for lenght in range(len_sequence):
            res = np.load(os.path.join(data_path,action,str(sequence),'{}.npy'.format(lenght)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [15]:
X = np.array(sequences)
Y = to_categorical(labels).astype(int) #one hot encoding the labels

In [16]:
#processing traing data
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size = 0.1)

In [17]:
#build and train the neural network
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

#tensor board allows to see the progress of your model while it is training
log_dir = os.path.join('logs')
call_back = TensorBoard(log_dir = log_dir)

In [18]:
model = Sequential()
model.add(LSTM(64, return_sequences = True, activation = 'relu',input_shape = (30,1662)))
model.add(LSTM(128, return_sequences = True, activation = 'relu'))
model.add(LSTM(64, return_sequences = False, activation = 'relu'))

model.add(Dense(64, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(actions.shape[0], activation = 'softmax'))

In [ ]:
model.compile(optimizer ='Adam',loss='categorical_crossentropy',metrics='categorical_accuracy')

In [ ]:
model.fit(X_train,y_train,epochs = 200, callbacks = [call_back])

In [ ]:
#making predictions
res_test = model.predict(X_test)
predicted_action = actions[np.argmax(res_test[1])]

In [ ]:
data_path = os.path.join('models') #path for exported data
model.save(os.path.join(data_path,'model{}.h5'.format(1))